<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/workflow_series_03_using_decorator_to_create_chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Decorator to Create Chunks Faster `#Agently_Workflow_Showcase_Series - 03`

## Demo Description

**Author:** Agently Team

**Series Link**:

[Last Document]: [02 - Using Condition to Branch Off](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_02_using_condition_to_branch_off.ipynb)

[Next Document]: [04 - Draw a Workflow Graph to Help You Observe the Workflow](https://github.com/Maplemx/Agently/blob/main/playground/workflow_series_04_draw_a_workflow_graph.ipynb)

**Description:**

Now you may have a basic concept about **Agently Workflow** like how to create chunks, how to connect chunks and how to start a workflow.

But some may say "It's still too complex to create a chunk." Can we just define a function and make it as a chunk? Because the main body of a chunk is basically the executor function.

Sure can do. Using decorator `@workflow.chunk()` provided by Agently workflow instance can make the chunk creation much more easier.

读到这个案例的时候，你可能已经对**Agently Workflow**有了很多基本的概念理解，比如：如何创建流程块（chunk），如何连接流程块，如何启动连接好的workflow等等。

不过可能还是有人会觉得，创建流程块这件事情是不是太复杂了？我们就不能直接把一个函数定义成流程块吗？因为从定义的视角看，一个流程块的主要内容就是它的执行函数。

当然没问题。我们在workflow实例里提供了`@workflow.chunk()`装饰器，使用它，就能更加连贯一体地结合函数定义创建流程块（chunk）了。

## Step 1: Install Packages

In [ ]:
!pip install -q -U Agently

> ℹ️ Agently Workflow is a new feature in version >= `3.2.0`

## Step 2: Rewrite Code in Showcase 02

### Code

In [7]:
import Agently
agent_factory = (
        Agently.AgentFactory()
        .set_settings("model.Google.auth.api_key", "")
        .set_settings("current_model", "Google")
)

# Step 0. Create Workflow Instance and Agent Instance
workflow = Agently.Workflow()
# We updated logger level settings framework logic.
# Now INFO log will not present unless you're in debug mode.
# So you don't need to reset logger level manually anymore when version >= 3.2.1.0
# workflow.settings.set("logger", logging.getLogger("Workflow").setLevel(logging.WARNING))
agent = agent_factory.create_agent()

# Step 1. Create Chunks
## Start Chunk
@workflow.chunk(
    # in @workflow.chunk() decorator, `chunk_id` is required
    chunk_id = "Start",
    # you can pass same params as `workflow.schema.create_chunk()` required except executor
    # if you did not state param `title`, it will using the value of `chunk_id` by default
    type = "Start"
)
# You don't have to decorate a function when the chunk type is "Start".

## User Input Chunk
@workflow.chunk(
    chunk_id = "User Input",
    handles = {
        "outputs": [{ "handle": "user_input" }],
    }
)
def user_input_executor(inputs, storage):
    return { "user_input": input("[User]: ") }

## Assistant Reply Chunk
@workflow.chunk(
    chunk_id = "Assistant Reply",
    handles = {
        "inputs": [{ "handle": "user_input" }],
        "outputs": [{ "handle": "assistant_reply" }],
    }
)
def assistant_reply_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    reply = (
        agent
            .chat_history(chat_history)
            .input(inputs["user_input"])
            .start()
    )
    print("[Assistant]: ", reply)
    return { "assistant_reply": reply }

## Update Chat History Chunk
@workflow.chunk(
    chunk_id = "Update Chat History",
    handles = {
        "inputs": [
            { "handle": "user_input" },
            { "handle": "assistant_reply" },
        ],
    },
)
def update_chat_history_executor(inputs, storage):
    chat_history = storage.get("chat_history") or []
    chat_history.append({ "role": "user", "content": inputs["user_input"] })
    chat_history.append({ "role": "assistant", "content": inputs["assistant_reply"] })
    storage.set("chat_history", chat_history)
    return


@workflow.chunk(
    chunk_id = "Goodbye",
)
def goodbye_executor(inputs, storage):
    print("Bye~👋")
    return

# Step 2. Connect Chunks in Orders
# using workflow.chunks["<chunk_id>"] instead of chunk variable
workflow.chunks["Start"].connect_to(workflow.chunks["User Input"])
(
    workflow.chunks["User Input"].handle("user_input")
        .if_condition(lambda data: data == "#exit").connect_to(workflow.chunks["Goodbye"])
        .else_condition().connect_to(workflow.chunks["Assistant Reply"].handle("user_input"))
)
workflow.chunks["User Input"].handle("user_input").connect_to(workflow.chunks["Update Chat History"].handle("user_input"))
workflow.chunks["Assistant Reply"].handle("assistant_reply").connect_to(workflow.chunks["Update Chat History"].handle("assistant_reply"))
workflow.chunks["Update Chat History"].connect_to(workflow.chunks["User Input"])

# Step 3. Start Workflow
workflow.start()

[User]: Hi, I'm Max
[Assistant]:  Hi Max, I'm Bard, a large language model trained by Google. Nice to meet you!
[User]: Yeah, tell me one tip about Rust
[Assistant]:  One tip about Rust is to use the borrow checker to your advantage. The borrow checker is a tool that helps you to ensure that your code is memory-safe. By understanding how the borrow checker works, you can write code that is more efficient and less error-prone. For example, you can use the borrow checker to help you to avoid data races and dangling pointers.
[User]: Tell me more about it
[Assistant]:  The Rust borrow checker is a tool that helps to ensure that your code is memory-safe. It does this by tracking the ownership and borrowing of data in your program. This helps to prevent data races and dangling pointers, which are two common sources of memory errors.

The borrow checker works by enforcing a set of rules about how data can be borrowed. For example, a value can only be borrowed once at a time, and a borrowed v

As you can see, the code works just fine as the code in last showcase did.

如您所见，上面这段修改过的代码和前一个案例中的代码运行效果一样好。

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI Agent Native application development](https://github.com/Maplemx/Agently)